In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve

os.chdir(r'F:\\KARTIK\\2021\\Freezing of Gait')

In [7]:
data_path = os.path.join(os.getcwd(), 'dataset_fog_release', 'dataset', 'FINAL FEATURES', 'final_ftrs_size_1.5_5.csv')
if os.path.exists(data_path):
    print(True)

True


In [8]:
df = pd.read_csv(data_path)
df.head()

,Unnamed: 0,0,Minimum_A_F,Maximum_A_F,RMS_A_F,Variance_A_F,STD_A_F,Kurtosis_A_F,Skewness_A_F,RangeA_F,...,RMS_A_L,Variance_A_L,RangeA_L,P_A_F,P_A_V,EN_A_F,EN_A_V,EN_A_L,ENt_A_F,ENt_A_V
0,3062,0,-3636,5080,1225.295373,1.445667e+06,1202.359139,6.835323,0.807714,8716,...,438.133934,158483.780300,2573,9.889245e+06,1.155377e+07,96086320,107843528,12285526,3.185094,3.069774
1,10661,0,-797,2545,678.327919,3.962067e+05,629.449525,4.757762,1.989022,3342,...,365.517270,110688.484400,2236,2.406776e+06,8.045909e+06,29448241,93522973,8550584,2.078001,1.191998
2,8155,0,-303,1181,250.297292,5.083236e+04,225.460340,9.921538,2.669784,1484,...,208.930423,19223.208740,643,2.948417e+05,4.324995e+06,4009519,67562434,2793723,2.780910,2.487021
3,3161,0,-161,40,48.628695,1.712905e+03,41.387260,1.300498,-1.003145,201,...,402.148332,1222.890625,159,6.431692e+04,3.831066e+06,151344,59428303,10350290,2.265934,2.466075
4,5727,0,-1878,1000,629.417489,3.679634e+05,606.599844,0.839529,-0.649126,2878,...,428.760951,143704.595500,2316,1.384678e+07,1.567011e+07,25354648,109820999,11765501,2.512256,1.454404


In [9]:
df['Energy_Mean'] = (df['EN_A_F'] + df['EN_A_V'] + df['EN_A_L'])/3
df = df.drop(['EN_A_F','EN_A_V','EN_A_L','Unnamed: 0'], axis = 1)

In [10]:
df.head()

,0,Minimum_A_F,Maximum_A_F,RMS_A_F,Variance_A_F,STD_A_F,Kurtosis_A_F,Skewness_A_F,RangeA_F,Minimum_A_V,...,RangeA_V,Minimum_A_L,RMS_A_L,Variance_A_L,RangeA_L,P_A_F,P_A_V,ENt_A_F,ENt_A_V,Energy_Mean
0,0,-3636,5080,1225.295373,1.445667e+06,1202.359139,6.835323,0.807714,8716,-196,...,4784,-1841,438.133934,158483.780300,2573,9.889245e+06,1.155377e+07,3.185094,3.069774,7.207179e+07
1,0,-797,2545,678.327919,3.962067e+05,629.449525,4.757762,1.989022,3342,127,...,1833,-1722,365.517270,110688.484400,2236,2.406776e+06,8.045909e+06,2.078001,1.191998,4.384060e+07
2,0,-303,1181,250.297292,5.083236e+04,225.460340,9.921538,2.669784,1484,813,...,373,-168,208.930423,19223.208740,643,2.948417e+05,4.324995e+06,2.780910,2.487021,2.478856e+07
3,0,-161,40,48.628695,1.712905e+03,41.387260,1.300498,-1.003145,201,872,...,196,316,402.148332,1222.890625,159,6.431692e+04,3.831066e+06,2.265934,2.466075,2.330998e+07
4,0,-1878,1000,629.417489,3.679634e+05,606.599844,0.839529,-0.649126,2878,19,...,2284,-1257,428.760951,143704.595500,2316,1.384678e+07,1.567011e+07,2.512256,1.454404,4.898038e+07


In [20]:
# Split the data for the classifier
y = df['0'].values
X = df.iloc[:,1:]
X = normalize(X)
X, y = shuffle(X, y, random_state = 2)

In [21]:
# Initiate the 10 - fold split 
kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 2)

# Initiate a random forest model
clf = RandomForestClassifier(
    n_estimators=500,
    criterion='gini',
    max_depth=50,
    min_samples_split=5,
    min_samples_leaf=15,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_split=None,
    bootstrap=True,
    n_jobs=-1,
    random_state=2,
    class_weight='balanced'
)
# split()  method generate indices to split data into training and test set.
for (train_index, test_index), i in zip(kf.split(X, y), range(10)):
    print(f'Fold:{i}, Train set: {len(train_index)}, Test set:{len(test_index)}')

Fold:0, Train set: 1988, Test set:221
Fold:1, Train set: 1988, Test set:221
Fold:2, Train set: 1988, Test set:221
Fold:3, Train set: 1988, Test set:221
Fold:4, Train set: 1988, Test set:221
Fold:5, Train set: 1988, Test set:221
Fold:6, Train set: 1988, Test set:221
Fold:7, Train set: 1988, Test set:221
Fold:8, Train set: 1988, Test set:221
Fold:9, Train set: 1989, Test set:220


In [22]:
score = cross_val_score(clf, X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

Scores for each fold are: [0.74208145 0.71945701 0.69230769 0.71040724 0.7239819  0.69230769
 0.69230769 0.760181   0.760181   0.73636364]
Average score: 0.72


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2, stratify = y, test_size = 0.40)

In [24]:
m = clf.fit(X_train, y_train)
y_pred = m.predict(X_test)
accuracy_score(y_test, y_pred)

0.7149321266968326

In [25]:
cnf = confusion_matrix(y_test, y_pred)
print(cnf)

[[ 98  36   8]
 [ 57 505  85]
 [  9  57  29]]


In [26]:
# For class pre-FoG
tp = cnf[1][1]
tn = cnf[0][0] + cnf[0][2] + cnf[2][0] + cnf[2][2]
fp = cnf[1][0] + cnf[1][2]
fn = cnf[0][1] + cnf[2][1]

In [27]:
precision = tp/(tp + fp)
recall =  tp / (tp + fn)    #sensitivity
F1_score = (2*tp) / (2*tp + fp + fn)

In [28]:
print("The evaluation metrics are:\nprecision = {},\nrecall = {}, \nF1-score = {}".format(precision, recall, F1_score) )

The evaluation metrics are:
precision = 0.7805255023183926,
recall = 0.8444816053511706, 
F1-score = 0.8112449799196787
